# __Tesi di Laurea – Visualizzazione spettrale dei grafi__

L’obiettivo è usare l'algebra spettrale per rappresentare graficamente la struttura di un grafo, sfruttando le informazioni che si ottengono dagli autovettori delle principali matrici associate al grafo (Laplaciana, normalizzata, Walk matrix, ecc.).

A seconda del tipo di grafo (orientato o meno), vengono fatte alcune ipotesi:
- Se il grafo è **non orientato**, si assume che sia **connesso**; altrimenti si lavora sulla componente connessa più grande.
- Se il grafo è **orientato**, si richiede che sia **fortemente connesso**, così da poter calcolare correttamente il **vettore di Perron** e costruire la **Laplaciana simmetrizzata** (secondo l’approccio di Chung).

Nel notebook si possono:
- caricare grafi da file (in formato `.json`)
- scegliere la matrice spettrale da usare
- calcolare autovalori e autovettori
- visualizzare il grafo in 1D o 2D usando gli autovettori come coordinate

Le funzioni sono state organizzate in modo modulare, per poter riutilizzare lo stesso codice con grafi diversi e con diversi tipi di rappresentazione.

## <hr style="border: 2px solid; margin-top: 15px;">
### __Librerie__

_Requisiti_

```
pip3 install -r requirements.txt
```

In [ ]:
#!pip3 install numpy networkx matplotlib ipympl notebook
import numpy as np
import visua
import networkx as nx
%matplotlib widget
np.set_printoptions(linewidth=250)

## <hr style="border: 2px solid; margin-top: 15px;">
### __Funzioni Ausiliarie__

### 
`ensure_connected(G)` si assicura che il grafo sia: 
1. connesso nel caso dei non orientati 
2. fortemente connesso nel caso degli orientati

###
`compute_directed_eig(G)`

1. Estrae da `G` la matrice di adiacenza `A`.  
2. Calcola per ogni nodo la somma degli archi in uscita (`degree`).  
3. Divide ogni riga di `A` per il grado corrispondente, ottenendo `P` (probabilità di passare da un nodo a un altro).  
4. **Autovettore di Perron**: Risolve `P.T * x = x` e ottiene il primo autovettore (`phi`), poi lo normalizza in modo che la somma sia 1.  
5. Crea matrici diagonali con la radice quadrata di `phi` (`Phi_sqrt`) e l’inverso della radice quadrata (`Phi_inv_sqrt`).  
6. **Matrice Laplaciana**: Crea la matrice Laplaciana: ***L = I - 0.5 • (Phi^1/2 • P • Phi^-1/2 + Phi^-1/2 • P.T • Phi^1/2)***.

###
`compute_eig(G, mat_type)`

1. **Connettività**  
   Verifica che il grafo sia connesso, con `ensure_connected(G)`.

2. **Grafi diretti**  
   Se il grafo è diretto e `mat_type == "laplacian"`, delega il calcolo a `compute_directed_eig(G)`.

3. **Grafi non diretti**  
   - **Matrice di adiacenza**: Converte il grafo in una matrice `A`.
   - **Vettore dei gradi**: Calcola `degrees` sommando gli archi in ogni riga di `A`.
   - **Matrice D**: Crea una matrice diagonale `D` con i gradi.
   - **Calcola gli autovettori**: a seconda del `mat_type` scelto calcola gli autovettori della matrice scelta (Laplaciana, Laplaciana normalizzata, Walk matrix, Lazy Walk matrix ecc.)

### Funzioni di Visualizzazione

Le tre funzioni seguenti permettono di rappresentare visivamente i grafi sulla base dei loro autovettori spettrali.

- `plot_line(G, mat_type, title)` → visualizza un autovettore alla volta (1D)
- `plot_2d(G, mat_type, x, y, title)` → visualizza i nodi in 2D usando due autovettori come coordinate, con possibilità di selezionare dinamicamente quali
- `plot_3d(G, mat_type, x, y, z, title)` → visualizza i nodi in 3D usando tre autovettori come coordinate, con possibilità di selezionare dinamicamente quali

Entrambe le funzioni calcolano internamente la matrice scelta tramite `compute_eig`, e poi disegnano il grafo.

### 
`plot_from_file(path, mat_type, mode, x, y, z, title)`

**Carica un grafo da file `.json` e richiama automaticamente la visualizzazione spettrale.**  
Permette di selezionare il tipo di matrice (`laplacian`, `normalized`, ecc.) e se visualizzarlo in 1D, 2D, 3D (tramite `mode="1d"`, `mode="2d"` o `mode="3d"`).  
Richiede solo il nome del file del grafo, nel formato node-link.

## <hr style="border: 2px solid; margin-top: 15px;">
###  __Esempi di utilizzo__

In questa sezione vengono mostrati alcuni esempi pratici di visualizzazione spettrale su grafi semplici e complessi.

L’obiettivo è verificare come le varie matrici (Laplaciana, normalizzata, walk matrix, ecc.) influenzano la rappresentazione dei nodi nel piano.

Gli esempi sono divisi per tipologia:
- Grafi noti e regolari (cammino, dodecaedro, griglia)
- Grafi casuali non orientati e orientati
- Grafi caricati da file `.json`

### 1. Cammino di 12 nodi

Costruzione di un cammino (path) di 12 nodi. Viene visualizzato un grafico a linea degli autovettori con uno slider per selezionare l’indice k.

In [ ]:
G1 = nx.path_graph(12)
fig, ax, slider, vals, vecs = visua.plot_line(G1, mat_type="laplacian", title="Cammino di 12 nodi")
# plt.close(fig)

### 2. Dodecaedro

Visualizzazione del dodecaedro usando la matrice Laplaciana.

In [ ]:
G2 = nx.dodecahedral_graph()
fig, ax, slider, vals, vecs = visua.plot_3d(G2, mat_type="laplacian", x=1, y=2, z=3, title="Dodecaedro")

### 3. Grafi non orientati

#### 
3.1 Albero Casuale

Visualizzazione di un albero casuale con la Laplaciana.

In [ ]:
G3 = nx.random_tree(20)
fig, ax, slider, vals, vecs = visua.plot_2d(G3, "laplacian", 1, 2, "Albero casuale")

In [ ]:
fig, ax, slider, vals, vecs = visua.plot_2d(G3, "laplacian", 1, 2, 3, "Albero casuale")

#### 
3.2 Grafo Non Orientato Casuale

Genera un grafo casuale con 20 nodi e 30 archi (assicurandosi che sia connesso) e lo visualizza con la Laplaciana normalizzata.

In [ ]:
n = 15
m = 25
while True:
    G4 = nx.gnm_random_graph(n, m)
    if nx.is_connected(G4):
        break
fig, ax, slider, vals, vecs = visua.plot_2d(G4, "normalized", 1, 2)

In [ ]:
fig, ax, slider, vals, vecs = visua.plot_3d(G4, "normalized", 1, 2, 3)

#### 
3.3 Griglia 4x4

In [ ]:
G6 = nx.grid_2d_graph(4, 4)   # Crea una griglia 4x4
G6 = nx.convert_node_labels_to_integers(G6)  # Rinomina i nodi in interi consecutivi
fig, ax, slider, vals, vecs = visua.plot_2d(G6, "laplacian", 1, 2, "Grid Graph")

In [ ]:
fig, ax, slider, vals, vecs = visua.plot_3d(G6, "laplacian", 1, 2, 3, "Grid Graph")

####
3.4 Cubo

In [ ]:
G7 = nx.cubical_graph()
fig, ax, slider, vals, vecs = visua.plot_3d(G7, "laplacian", 1, 2, 3, "Cubo")

####
3.5 Ipercubo

In [ ]:
G8 = nx.hypercube_graph(4)
fig, ax, slider, vals, vecs = visua.plot_2d(G8, "walknorm", 1, 2, "Ipercubo")

In [ ]:
fig, ax, slider, vals, vecs = visua.plot_3d(G8, "normalized", 1, 2, 3, "Ipercubo")

####
3.6 Grafo Casuale Regolare

In [ ]:
G9 = nx.random_regular_graph(d=4, n=20)
fig, ax, slider, vals, vecs = visua.plot_2d(G9, "normalized", 1, 2, "Grafo casuale regolare")

### 4. Grafi orientati

Per grafi orientati viene usata la procedura con P e il vettore di Perron per calcolare la Laplaciana simmetrica.

In [ ]:
G5 = nx.gnp_random_graph(30, 0.3, directed=True)
fig, ax, slider, vals, vecs = visua.plot_2d(G5, "laplacian", 1, 2)

### 5. Caricamento con JSON
Il grafo del logo di Yale mantiene una struttura regolare e simmetrica.

In [ ]:
fig, ax, _, _, _ = visua.plot_from_file("graphs/yale_graph.json", mat_type="laplacian", mode="2d", x=1, y=2, weight="weight")       

In [ ]:
fig, ax, slider, vals, vecs = visua.plot_from_file("graphs/yale_graph.json", "laplacian", mode="3d", x=1, y=2, z=3, weight="weight")